In [1]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"
database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [2]:
graph.query('''
MATCH (n:__Entity__)
WHERE n.description IS NOT NULL and apoc.meta.cypher.type(n.description) <> 'STRING'
REMOVE n.embedding
SET n.description = apoc.text.join(n.description, ", ")
RETURN n.description
''')

[{'n.description': '至郵局劃撥至本公司各地分公司之劃撥帳號, 至郵局劃撥至本公司之郵政劃撥帳號'},
 {'n.description': '僅適用業務通路, 行動保全續期保費的服務說明，僅適用業務通路'},
 {'n.description': '輸入首期保險費金額按送出, 輸入後按送出'},
 {'n.description': '確認交易金額, 進入 e-Bill 全國繳費網→確認交易金額及繳費方式'}]

In [3]:
graph.query(
    """
MATCH (n:`__Community__`)<-[:IN_COMMUNITY]-()<-[:HAS_ENTITY]-(c)
WITH n, count(distinct c) AS chunkCount
SET n.weight = chunkCount"""
)

[]

In [4]:
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
    azure_endpoint='https://sales-chatbot-llm.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-3-small',
    openai_api_version='2023-05-15'
)

In [5]:
# from langchain_community.vectorstores import Neo4jVector
# # ! pip3 install -U langchain-huggingface
# import os
# os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/storage/models/embedding_models'
# from langchain_huggingface import HuggingFaceEmbeddings
# # Choose from https://huggingface.co/spaces/mteb/leaderboard

# # embedding = HuggingFaceEmbeddings(model_name="lier007/xiaobu-embedding-v2")

# model_path = os.path.join(os.environ['SENTENCE_TRANSFORMERS_HOME'], 'models--lier007--xiaobu-embedding-v2/snapshots/ee0b4ecdf5eb449e8240f2e3de2e10eeae877691')
# embedding = HuggingFaceEmbeddings(model_name=model_path)

In [6]:
# node_label='__Entity__'
# embedding_node_property='embedding'
# fetch_query = (
#     f"MATCH (n:`{node_label}`) "
#     f"WHERE n.{embedding_node_property} IS null "
#     "AND any(k in $props WHERE n[k] IS NOT null) "
#     f"RETURN elementId(n) AS id, reduce(str='',"
#     "k IN $props | str + '\\n' + k + ':' + coalesce(n[k], '')) AS text "
#     "LIMIT 1000"
# )
# datas = graph.query(fetch_query, params={"props": ['id', 'description']})
# datas

In [7]:
# import sys
# sys.path.append('..')
# from tools.TokenCounter import num_tokens_from_string

# tokens_num = 0
# for data in datas:
#     tokens_num += num_tokens_from_string(data['text'])
# tokens_num

In [24]:
from langchain_community.vectorstores import Neo4jVector

lc_retrieval_query = """
WITH collect(node) as nodes
// Entity - Text Unit Mapping
WITH
collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)<-[:HAS_CHILD]->(p:__Parent__)
    WITH c, p, count(distinct n) as freq
    RETURN p.content AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
// Entity - Report Mapping
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WHERE c.summary is not null
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
// Outside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE NOT m IN nodes and r.description is not null
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topOutsideRels
} as outsideRels,
// Inside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE m IN nodes and r.description is not null
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
// Entities description
collect {
    UNWIND nodes as n
    match (n)
    WHERE n.description is not null
    RETURN n.description AS descriptionText
} as entities
// We don't have covariates or claims here
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: outsideRels + insideRels, 
       Entities: entities} AS text, 1.0 AS score, {} AS metadata
"""

vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="embedding",
                                    node_label='__Entity__', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['id', 'description'],
                                    retrieval_query=lc_retrieval_query)

In [25]:
topChunks = 3
topCommunities = 3
topOutsideRels = 10
topInsideRels = 10
topEntities = 10

In [26]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

In [30]:
docs = vectorstore.similarity_search(
    "首期匯款帳號有哪些銀行, 匯款帳號是什麼?",
    k=topEntities,
    params={
        "topChunks": topChunks,
        "topCommunities": topCommunities,
        "topOutsideRels": topOutsideRels,
        "topInsideRels": topInsideRels,
    },
)
print(docs[0].page_content)

Entities:
- 首期保險費採自行匯款者，可透過銀行/郵局以匯款/劃撥方式繳交，並可利用e-bill首期立即繳費及中信ATM繳費。
- 匯款銀行
- 匯款銀行
- 匯款銀行
- 保戶可透過全國繳費網平台繳納首期保險費，可以「非約定帳戶」或「晶片金融卡」 (網路ATM)繳費。僅開放台幣保單且繳款人應為要保人
- 無法授權扣款
Reports:
- 在同一社區中，有三個節點，分別是「四、 首期保險費-採自行匯款管道」、「(二) 外幣-首期匯款帳號」和「(一) 台幣-首期匯款帳號【業務通路適用】」。其中，「四、 首期保險費-採自行匯款管道」包含了「(二) 外幣-首期匯款帳號」和「(一) 台幣-首期匯款帳號【業務通路適用】」這兩個子節點。
- 在這個社區中，有三個主要概念節點和兩個關係。首先，「要保人與授權人不同一人」這個概念強調電話行銷人員需要取得授權人簽名的紙本「保險費信用卡付款授權書」的正本或影本。其次，「再行確認之機制」這個概念涉及指派非銷售人員以電話回訪方式確認授權人的各種信息，並向銀行驗證這些信息的正確性，經銀行驗證後才向銀行請款並寄發保單。最後，「首期保險費-採自行匯款作業」這個概念說明首期保險費可以通過銀行或郵局以匯款或劃撥方式繳交，並可利用e-bill或中信ATM進行繳費。

在關係方面，「再行確認之機制」與「首期保險費-採自行匯款作業」和「要保人與授權人不同一人」這兩個概念相關聯，顯示出再行確認機制在保險費繳交和授權人確認過程中的重要性。
- 在同一社區中，有三個節點，分別是「四、 首期保險費-採自行匯款管道」、「(二) 外幣-首期匯款帳號」和「(一) 台幣-首期匯款帳號【業務通路適用】」。其中，「四、 首期保險費-採自行匯款管道」包含了「(二) 外幣-首期匯款帳號」和「(一) 台幣-首期匯款帳號【業務通路適用】」這兩個子節點。
Chunks:
- 。  ### 相關規定  1. **金融機構轉帳**    - 使用金融機構轉帳，需待核印成功(另新契約需待核保通過)後，始進行扣款作業。    - 請務必親視並確認授權書各項資料、簽名、印章及帳號填寫完整及正確。  2. **信用卡繳費**    - 使用信用卡，信用卡卡號及授權人身分證統一編號須送「信用卡輔助持卡人身分驗證平台」，經驗證成功始得進行首期保險費信用卡扣款作業。    - 首期使用信用卡

In [30]:
docs

[Document(page_content='Entities:\n- 首期保險費是保戶首次繳交的保險費用，採用金融機構轉帳作業。\n- 因核印失敗、請款失敗或主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。\n- 依各險種投保規則辦理\n- 轉帳扣款作業需經核印成功且同意承保後始得進行。\n- 保戶指的是與保險公司簽約的個人或團體。\n- 需填寫授權書，繳款人為非保單關係人時須檢附關係證明文件。相關規定請參照繳費方式及受理應檢附文件。\n- 不得為空白授權，應依是否取得保單號碼填寫相關欄位，未填寫完整須重新檢附授權書\n- 核印和扣款時間需參照各金融機構的相關規定。\n- 需至各銀行臨櫃留存印鑑，核印成功後便能授權扣款。\n- 須留存金融機構之印鑑或簽章樣式\nReports:\n- 在同一社區中，有兩個重要的節點和一個關鍵的關係。節點包括：\n\n1. 一個沒有特定類型的節點，描述的是「新契約首期保險費繳費管道異動之繳費日認定原則」。\n2. 一個類型為「概念」的節點，描述的是「新契約」。具體內容為：因核印失敗、請款失敗或主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理。\n\n這兩個節點之間存在一個關係：「新契約」依照「新契約首期保險費繳費管道異動之繳費日認定原則」來進行生效日認定。\n- None\n- 在同一個社區中，有以下的資訊：\n\n1. 要保人（Person）：在授權書要保人欄位內須簽章，並且簽章需與要保書相同。\n2. 保險費暨保險單借款利息自動轉帳付款授權書（Document）：需要填寫授權書，當繳款人非保單關係人時，需附上關係證明文件，詳情請參照繳費方式及應檢附文件。\n3. 授權人（Person）：需留存金融機構的印鑑或簽章樣式。\n\n關係如下：\n- 保險費暨保險單借款利息自動轉帳付款授權書需要授權人，授權人需留存金融機構的印鑑或簽章樣式。\n- 保險費暨保險單借款利息自動轉帳付款授權書需要要保人，要保人需在授權書要保人欄位簽章，且簽章需與要保書相同。\nChunks:\n- ，經核印成功 (且 同意承保 )始得進行首期保險費 轉帳扣款作業 。核印、扣款時間 請參照各金融機構核印、 扣款時間及相關規定 。  5. 數位帳戶 請至各銀行臨櫃留存印鑑、核印成功便